In [1]:
import matplotlib.pyplot as plt
import numpy as np
from neurolib.models.multimodel import MultiModel
from neurolib.models.multimodel.builder import HopfNetwork
from neurolib.utils.functions import getPowerSpectrum
from neurolib.utils.stimulus import ZeroInput

from hopf_cfc_network import SlowFastHopfNetwork
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.exploration import BoxSearch

import h5py

plt.style.use("seaborn")

# w instrinsic
# 0.003 ~ 0.5Hz
# 0.005 ~ 0.8Hz
# 0.01 ~ 2Hz
# 0.05 ~ 8Hz
# 0.06 ~ 10Hz
# 0.08 ~ 12Hz
# 0.1 ~ 17Hz
# 0.2 ~ 32Hz
# 0.3 ~ 50Hz

In [2]:
def measure_coupling_eval(traj):

    pars = search.getParametersFromTraj(traj)
    n_bins = 18

    phase_slow = np.arctan2(pars['y'][0],pars['x'][0])
    amp_fast = np.sqrt(pars['x'][1] ** 2 + pars['y'][1] ** 2)
    binned_phase = np.digitize(phase_slow,bins=np.linspace(-np.pi,np.pi,n_bins+1))
    mean_bin_amp = np.zeros(n_bins+1) # in theory index of bins goes from 0 to N_BINS
    for bin_idx in np.unique(binned_phase):
        mean_bin_amp[bin_idx] = np.mean(amp_fast[binned_phase==bin_idx])

    mean_bin_amp = mean_bin_amp[1:] # because in theory there could be stuff that is smaller than -pi, then actually the interval between -pi and the next bin has index 1.
    #normalize the mean amplitude in each bin
    mean_bin_amp = mean_bin_amp/np.sum(mean_bin_amp)
    mi = modulation_index(mean_bin_amp)

    result_dict = {"modulation_index": mi}
    search.saveToPypet(result_dict, traj)

def modulation_index(P):
    """
    MI = KL(P,U)/log(n), where U is a uniform distribution.
    :param P: mean amplitude for each phase bin, shape (N_BINS,)
    :return modulation index, quantitative measure of phase amplitude coupling
    """
    n_bins = P.shape[0]
    try:
        assert np.isclose(np.sum(P),1)
    except AssertionError:
        print("Please normalize P!")
    kl = np.log(n_bins) + np.sum(P*np.log(P))
    return kl/np.log(n_bins)

How exactly do I do the parameter exploration passing both the model and the evaluation function?

In [3]:
model = MultiModel(SlowFastHopfNetwork())

In [4]:
parameters = ParameterSpace({"slow_to_fast_connection": np.linspace(0,0.6, 5)})

search = BoxSearch(model = model,evalFunction = measure_coupling_eval , parameterSpace = parameters, filename="Hopf_conn_test.hdf")

MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `./data/hdf/Hopf_conn_test.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.


AssertionError: 